# 外汇意图

创建一个比如 dual_thrust 更好的策略

## dual_thrust策略概述

1. 根据之前N个bar的ohlc 组合后确认价格中线: Mprice

2. 基于中线, 根据之前N个bar的ohlc 组合后, 设定上下轨: Hprice, Lprice

3. 开仓策略: 当前价格 突破 Mprice, 开仓

4. 平仓策略: 

        4.1 当前价格 突破 上下轨后 回归 中线, 平仓(简单止损)
        4.2 当前价格 破前N个kbar 最高最低价组合, 平仓(简单止赢)
    
5. 开仓后价格中线不移动





In [39]:
# 确认中线
dual_params={
    'resample' : '05T', 
    'barNum' : 1,
    'file_path': './kam_hsi.csv',
    'check_date': '2019-09-26'
}

dual_params['resample']

'05T'

In [40]:
# 研究文档见同目录下研究笔记, 更新文档, 收集代码块都放 研究笔记

# from jqdatasdk import *
# auth('13817230201','JQ2melx2018')


# 加载自定义库, 日常数据操控管理模块

import pandaAI.robot as robot

import imp
imp.reload(robot)



# 数据处理
import pandas as pd
import numpy as np
import copy
import datetime,pickle

from datetime import datetime
from datetime import timedelta

# 时间序列特征生成包
# import tsfresh
# from tsfresh import extract_features, extract_relevant_features, select_features

# 区间数处理 
# zoom2_5= Interval(2,5)
# 2 in zoom2_5
#from interval import Interval 

# 机器学习
import sklearn
import sklearn.metrics as me

# 目标函数, error 越小越好, score 越大约好
from sklearn.metrics import log_loss, f1_score, mean_absolute_error,mean_squared_error,r2_score,accuracy_score,roc_auc_score, balanced_accuracy_score
            
from sklearn.model_selection import train_test_split # 数据集划分
# from sklearn.ensemble import RandomForestClassifier as RM#随机森林分类模型
# import xgboost as xgb

# from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing 
import seaborn as sns
import matplotlib.pyplot as plt #可视化模块

from sklearn.preprocessing import StandardScaler
# This import registers the 3D projection, but is otherwise unused.
#from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import talib as ta  #金融数据计算

# from sklearn.lda import LDA #线性判别分析法，返回降维后的数据

# %pylab inline
# from gplearn.genetic import SymbolicRegressor, SymbolicTransformer


In [41]:
# 获取数据
df1 = pd.read_csv(dual_params['file_path'],  skiprows=1, header=None, sep=',', names=['dates', 'open', 'high', 'low', 'close','volume', 'barcout','avg'])
#df1 = pd.read_csv('./data/DAT_ASCII_USDJPY_M1_201902.csv', sep=';', names=['dates', 'open', 'high', 'low', 'close','volume', 'barcout','avg'])

df1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5035 entries, 3298 to 1300
Data columns (total 8 columns):
dates      5035 non-null object
open       5035 non-null float64
high       5035 non-null float64
low        5035 non-null float64
close      5035 non-null float64
volume     5035 non-null int64
barcout    5035 non-null int64
avg        5035 non-null float64
dtypes: float64(5), int64(2), object(1)
memory usage: 354.0+ KB


In [42]:
# 整理数据
import matplotlib.dates as mdates

mm = robot.MM()

#设置索引
df2 = mm.set_index(df1,cname='dates')

period_data = df2.resample(dual_params['resample']).last()
# 分别对于开盘、收盘、最高价、最低价进行处理
period_data['open'] = df2['open'].resample(dual_params['resample']).first()
# 处理最高价和最低价
period_data['high'] = df2['high'].resample(dual_params['resample']).max()
# 最低价
period_data['low'] = df2['low'].resample(dual_params['resample']).min()
# 成交量 这一周的每天成交量的和
period_data['volume'] = df2['volume'].resample(dual_params['resample']).sum()

df3 = period_data.dropna(axis=0) # 缺失值处理

df3['dates2'] = df3.index
df3['dates2'] = df3['dates2'].apply(mdates.date2num)
df3.head(6)


                         open      high       low     close  volume  barcout  \
dates                                                                          
2019-09-26 15:50:00  26027.30  26041.44  26019.26  26028.00       0        0   
2019-09-26 15:45:00  26022.06  26036.74  26016.45  26032.68       0        0   
2019-09-26 15:40:00  26012.17  26026.12  26007.72  26025.37       0        0   
2019-09-26 15:35:00  26005.39  26022.31  26000.44  26012.17       0        0   
2019-09-26 15:30:00  25990.16  26012.34  25984.56  26003.89       0        0   

                       avg  
dates                       
2019-09-26 15:50:00  425.0  
2019-09-26 15:45:00  350.0  
2019-09-26 15:40:00  174.0  
2019-09-26 15:35:00  287.0  
2019-09-26 15:30:00  252.0  


//anaconda3/envs/fxtrader/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/envs/fxtrader/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,open,high,low,close,volume,barcout,avg,dates2
dates,,,,,,,,
2019-06-11 14:20:00,27835.27,27851.03,27831.32,27846.18,0,0.0,169.0,737221.597222
2019-06-11 14:25:00,27846.18,27849.67,27832.00,27838.70,0,0.0,230.0,737221.600694
2019-06-11 14:30:00,27838.70,27840.84,27822.60,27830.88,0,0.0,203.0,737221.604167
2019-06-11 14:35:00,27830.88,27842.41,27817.26,27827.84,0,0.0,235.0,737221.607639
2019-06-11 14:40:00,27827.84,27834.31,27817.89,27830.58,0,0.0,197.0,737221.611111
2019-06-11 14:45:00,27830.58,27833.96,27807.86,27823.32,0,0.0,233.0,737221.614583


In [43]:

df3 = df3.dropna(axis=0) # 缺失值处理
df3.head()



,open,high,low,close,volume,barcout,avg,dates2
dates,,,,,,,,
2019-06-11 14:20:00,27835.27,27851.03,27831.32,27846.18,0,0.0,169.0,737221.597222
2019-06-11 14:25:00,27846.18,27849.67,27832.00,27838.70,0,0.0,230.0,737221.600694
2019-06-11 14:30:00,27838.70,27840.84,27822.60,27830.88,0,0.0,203.0,737221.604167
2019-06-11 14:35:00,27830.88,27842.41,27817.26,27827.84,0,0.0,235.0,737221.607639
2019-06-11 14:40:00,27827.84,27834.31,27817.89,27830.58,0,0.0,197.0,737221.611111


In [44]:
df3.tail(1)

,open,high,low,close,volume,barcout,avg,dates2
dates,,,,,,,,
2019-09-26 15:50:00,26027.3,26041.44,26019.26,26028.0,0,0.0,425.0,737328.659722


In [57]:
# 基于线性回归的模型加载

from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

# get buy_base
reg_buy_open = joblib.load('reg_buy_open.pkl')

# get buy_high
reg_buy_break = joblib.load('reg_buy_break.pkl')

# get buy_base
reg_sale_open = joblib.load('reg_sale_open.pkl')

# get buy_base
reg_sale_break = joblib.load('reg_sale_break.pkl')

scale = joblib.load('scale.pkl')


In [58]:
df3.iloc[-3:-1][['open','high','low','close']]

,open,high,low,close
dates,,,,
2019-09-26 15:40:00,26012.17,26026.12,26007.72,26025.37
2019-09-26 15:45:00,26022.06,26036.74,26016.45,26032.68


In [ ]:
#开仓策略: 当前价格 突破 Mprice, 开仓
# 使用5分钟bar, 但是每分钟重新获取一次数据, 预期是last bar 会因当前价格变化而变化

# init 初始化
panda_robot = PandaRobot.new(balance=0, position_state='empty', position=0)

# 这里开始循环获取当前ohlc 
current_price = online_data.tail(1)['close']
pre_data = online_data.iloc[-2:-1][['open','high','low','close']] # 倒数第二个bar作为数据预测基准, 因为当前bar还没有走完

if panda_robot.position_state() == 'empty':  
    
    if current_price > reg_buy_open(pre_data):
        panda_robot.buy_open() # 默认100% 全部

    if current_price < reg_sale_open(pre_data):
        panda_robot.sale_open() # 默认100% 全部


        
# 需要创建一个 panda_robot 类
# TODO: position_state = buy|sale|empty
if panda_robot.position_state() == 'buy':  
    # 冲高回落, 平仓
    if online_data.tail(1)['high'] > reg_buy_break.predict(pre_data).last and current_price < reg_buy_open.predict(pre_data).last:
        panda_robot.close_position()
    # 移动平仓    
    if current_price <  pre_data['close'].last:
        panda_robot.close_position()
    
        
if panda_robot.position_state() == 'sale'        
    if online_data.tail(1)['low'] > reg_buy_break.predict(pre_data).last and current_price > reg_buy_open.predict(pre_data).last :
        panda_robot.close_position()
        
    if current_price >  pre_data['close'].last:
        panda_robot.close_position()

        

In [220]:
panda_robot.trade_history() # 打印交易记录
panda_robot.analysis() # 打印统计分析, 交易次数, 连续盈利次数, 连续亏损次数, 最大亏损点差, 最大盈利点差, 开多, 开空盈利汇总


,open,high,low,close
dates,,,,
2019-09-26 15:45:00,26022.06,26036.74,26016.45,26032.68
